<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.4: A FIRRTL Transform Example

**Prev: [Common Pass Idioms](4.3_firrtl_common_idioms.ipynb)**<br>

这个 AnalyzeCircuit Transform 遍历一个 `firrtl.ir.Circuit`，并记录它在每个模块中找到的加法操作数量。

## Setup

请运行以下代码：

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
// Compiler Infrastructure

// Firrtl IR classes

// Map functions

// Scala's mutable collections
import scala.collection.mutable

## Counting Adders Per Module

如前所述，Firrtl 电路使用树形表示法表示：
  - Firrtl `Circuit` 包含一个 `DefModule` 序列。
  - `DefModule` 包含一个 `Port` 序列，以及可能包含一个 `Statement`。
  - `Statement` 可以包含其他 `Statement` 或 `Expression`。
  - `Expression` 可以包含其他 `Expression`。

要访问电路中的所有 Firrtl IR 节点，我们编写递归遍历此树的函数。为了记录统计信息，我们将传递一个 `Ledger` 类，并在遇到加法操作时使用它：

In [ ]:
class Ledger {
  import firrtl.Utils
  private var moduleName: Option[String] = None
  private val modules = mutable.Set[String]()
  private val moduleAddMap = mutable.Map[String, Int]()
  def foundAdd(): Unit = moduleName match {
    case None => sys.error("Module name not defined in Ledger!")
    case Some(name) => moduleAddMap(name) = moduleAddMap.getOrElse(name, 0) + 1
  }
  def getModuleName: String = moduleName match {
    case None => Utils.error("Module name not defined in Ledger!")
    case Some(name) => name
  }
  def setModuleName(myName: String): Unit = {
    modules += myName
    moduleName = Some(myName)
  }
  def serialize: String = {
    modules map { myName =>
      s"$myName => ${moduleAddMap.getOrElse(myName, 0)} add ops!"
    } mkString "\n"
  }
}

现在，让我们定义一个 FIRRTL 变换，它遍历电路，并在遇到加法器（`DoPrim` 的 `op` 参数为 `Add`）时更新我们的 `Ledger`。暂时不用担心 `inputForm` 或 `outputForm`。

花点时间来理解 `walkModule`、`walkStatement` 和 `walkExpression` 如何在 FIRRTL AST 中遍历所有的 `DefModule`、`Statement` 和 `Expression` 节点。

需要回答的问题：
  - **为什么 `walkModule` 不调用 `walkExpression`？**
  - **为什么 `walkExpression` 进行后序遍历？**
  - **你能修改 `walkExpression` 以进行表达式的前序遍历吗？**

In [ ]:
class AnalyzeCircuit extends firrtl.Transform {
  import firrtl._
  import firrtl.ir._
  import firrtl.Mappers._
  import firrtl.Parser._
  import firrtl.annotations._
  import firrtl.PrimOps._
    
  // Requires the [[Circuit]] form to be "low"
  def inputForm = LowForm
  // Indicates the output [[Circuit]] form to be "low"
  def outputForm = LowForm

  // Called by [[Compiler]] to run your pass. [[CircuitState]] contains
  // the circuit and its form, as well as other related data.
  def execute(state: CircuitState): CircuitState = {
    val ledger = new Ledger()
    val circuit = state.circuit

    // Execute the function walkModule(ledger) on every [[DefModule]] in
    // circuit, returning a new [[Circuit]] with new [[Seq]] of [[DefModule]].
    //   - "higher order functions" - using a function as an object
    //   - "function currying" - partial argument notation
    //   - "infix notation" - fancy function calling syntax
    //   - "map" - classic functional programming concept
    //   - discard the returned new [[Circuit]] because circuit is unmodified
    circuit map walkModule(ledger)

    // Print our ledger
    println(ledger.serialize)

    // Return an unchanged [[CircuitState]]
    state
  }

  // Deeply visits every [[Statement]] in m.
  def walkModule(ledger: Ledger)(m: DefModule): DefModule = {
    // Set ledger to current module name
    ledger.setModuleName(m.name)

    // Execute the function walkStatement(ledger) on every [[Statement]] in m.
    //   - return the new [[DefModule]] (in this case, its identical to m)
    //   - if m does not contain [[Statement]], map returns m.
    m map walkStatement(ledger)
  }

  // Deeply visits every [[Statement]] and [[Expression]] in s.
  def walkStatement(ledger: Ledger)(s: Statement): Statement = {

    // Execute the function walkExpression(ledger) on every [[Expression]] in s.
    //   - discard the new [[Statement]] (in this case, its identical to s)
    //   - if s does not contain [[Expression]], map returns s.
    s map walkExpression(ledger)

    // Execute the function walkStatement(ledger) on every [[Statement]] in s.
    //   - return the new [[Statement]] (in this case, its identical to s)
    //   - if s does not contain [[Statement]], map returns s.
    s map walkStatement(ledger)
  }

  // Deeply visits every [[Expression]] in e.
  //   - "post-order traversal" - handle e's children [[Expression]] before e
  def walkExpression(ledger: Ledger)(e: Expression): Expression = {

    // Execute the function walkExpression(ledger) on every [[Expression]] in e.
    //   - return the new [[Expression]] (in this case, its identical to e)
    //   - if s does not contain [[Expression]], map returns e.
    val visited = e map walkExpression(ledger)

    visited match {
      // If e is an adder, increment our ledger and return e.
      case DoPrim(Add, _, _, _) =>
        ledger.foundAdd
        e
      // If e is not an adder, return e.
      case notadd => notadd
    }
  }
}

## 运行我们的 Transform

现在我们已经定义了它，让我们在一个 Chisel 设计上运行它！首先，让我们定义一个 Chisel 模块。

In [ ]:
// Chisel stuff
import chisel3._
import chisel3.Input // Technicality: avoid a conflict with _root_.almond.input.Input
import chisel3.util._

In [ ]:
class AddMe(val nInputs: Int, val width: Int) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Vec(nInputs, UInt(width.W)))
    val out = Output(UInt(width.W))
  })
  io.out := io.in.reduce(_ +& _)
}

Next, let's elaborate it into FIRRTL AST syntax.

In [ ]:
val firrtlSerialization = chisel3.Driver.emit(() => new AddMe(8, 4))


最后，让我们将我们的 FIRRTL 编译成 Verilog，但在编译中包含我们自定义的变换。注意，它会打印出它找到的加法操作的数量！

**注意**（2021年1月）：由于[一个bug](https://github.com/freechipsproject/chisel-bootcamp/issues/129)，以下行可能会失效。

In [ ]:
val verilog = compileFIRRTL(firrtlSerialization, new firrtl.VerilogCompiler(), Seq(new AnalyzeCircuit()))

`compileFIRRTL` 函数仅在本教程中定义 - 在后面的部分中，我们将描述如何插入自定义变换的过程。

本节内容就到这里！